In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install git+https://github.com/ageitgey/face_recognition

In [ ]:
import os
import cv2
import face_recognition
from skimage.feature import hog
from skimage import exposure
from google.colab.patches import cv2_imshow
import numpy as np

# Set up paths for known and unknown faces
known_faces_dir = '/content/gdrive/MyDrive/images/known_faces/'
unknown_faces_dir = '/content/gdrive/MyDrive/images/unknown_faces/'

# Load known faces
known_faces = []
known_face_names = []
for filename in os.listdir(known_faces_dir):
  if os.path.isfile(os.path.join(known_faces_dir, filename)):
     name = os.path.splitext(filename)[0]
     image = face_recognition.load_image_file(known_faces_dir + filename)
     try:
         # Normalize the image
         image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
         encoding = face_recognition.face_encodings(image)[0]
         known_faces.append(encoding)
         known_face_names.append(name)
         print(f"face detected in {filename}")
     except IndexError:
         print(f"No face detected in {filename}")

# Process unknown faces
for filename in os.listdir(unknown_faces_dir):
    image = face_recognition.load_image_file(unknown_faces_dir + filename)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect face locations using HOG algorithm
    face_locations = face_recognition.face_locations(gray_image, model='hog')

    # Calculate feature vector using HOG algorithm
    features, hog_image = hog(gray_image, orientations=8, pixels_per_cell=(16, 16), cells_per_block=(1, 1), visualize=True, multichannel=False)

    # Adjust the gamma value for exposure control
    gamma = 0.38  # adjust as needed
    gamma_corrected = np.uint8(cv2.pow(hog_image / 255., gamma) * 255.)


    # Show the feature vector
    print(f"Feature vector for {filename}: {features}")

    # Show the local gradient image
    print(f"HOG IMAGE for {filename}")
    cv2_imshow(gamma_corrected)
    cv2.waitKey(0)



In [ ]:
# Loop through the unknown face images and extract their encodings
for filename in os.listdir(unknown_faces_dir):
    image = face_recognition.load_image_file(unknown_faces_dir + filename)
    encodings = face_recognition.face_encodings(image)
    if len(encodings) > 0:
        # Print the 128-dimensional face encoding to the console
        print(f"128-dimensional face encoding for {filename}")
        print(encodings[0])

In [ ]:
from PIL import Image, ImageDraw

# Loop over all images in the unknown faces directory
for filename in os.listdir(unknown_faces_dir):
    # Load the image file
    image = face_recognition.load_image_file(unknown_faces_dir + filename)

    # Find all facial features in the image
    face_landmarks_list = face_recognition.face_landmarks(image)

    # Convert the image to PIL format
    pil_image = Image.fromarray(image)

    # Create a draw object
    draw = ImageDraw.Draw(pil_image)

    # Loop over each facial feature and draw them on the image
    for face_landmarks in face_landmarks_list:
        for name, points in face_landmarks.items():
            draw.line(points, width=2)

    # Show the image with the facial landmarks
    print(f"Landmark Image for {filename}")
    pil_image.show()

In [ ]:
from sklearn import svm

# Load the known faces and labels
known_faces = []
labels = []
for filename in os.listdir(known_faces_dir):
    image = face_recognition.load_image_file(known_faces_dir + filename)
    try:
        # Normalize the image
        image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
        encoding = face_recognition.face_encodings(image)[0]
        known_faces.append(encoding)
        known_face_names.append(name)
        labels.append(os.path.splitext(filename)[0])
        print(f"face detected in {filename}")
    except IndexError:
        print(f"No face detected in {filename}")

# Train the SVM classifier
svm_clf = svm.LinearSVC()
svm_clf.fit(known_faces, labels)

# Loop through the unknown faces and compare them to the known faces
for filename in os.listdir(unknown_faces_dir):
    image = face_recognition.load_image_file(unknown_faces_dir + filename)
    face_encodings = face_recognition.face_encodings(image)
    if len(face_encodings) == 0:
        print(f"No face detected in {filename}")
    else:
        face_encoding = face_encodings[0]
        is_match = svm_clf.predict([face_encoding])[0]
        if is_match:
            print(f"The person in {filename} is {is_match}.")
        else:
            print(f"The person in {filename} is not recognized.")


In [ ]:
# Loop through the unknown faces
for filename in os.listdir(unknown_faces_dir):
    image = face_recognition.load_image_file(unknown_faces_dir + filename)
    face_encodings = face_recognition.face_encodings(image)
    if len(face_encodings) == 0:
        print(f"No face detected in {filename}")
    else:
        print(f"Face detected in {filename}")
        face_encoding = face_encodings[0]

        # Compare the unknown face to each of the known faces
        distances = face_recognition.face_distance(known_faces, face_encoding)

        # Print the Euclidean distances between the unknown face and the known faces
        print(f"Distances for {filename}: {distances}")

In [ ]:
# Loop through all the unknown faces and recognize them
for filename in os.listdir(unknown_faces_dir):
    image = face_recognition.load_image_file(unknown_faces_dir + filename)
    # Normalize the image
    image = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
    face_locations = face_recognition.face_locations(image)
    face_encodings = face_recognition.face_encodings(image, face_locations)
    
    # Convert the image from RGB to BGR for OpenCV
    rgb_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Loop through each face found in the unknown image
    for (top, right, bottom, left), unknown_encoding in zip(face_locations, face_encodings):
        # See if the face is a match for any known faces
        matches = face_recognition.compare_faces(known_faces, unknown_encoding)
        
        # Find the index of the first match in the known_faces array
        match_index = -1
        for i in range(len(matches)):
            if matches[i]:
                match_index = i
                break
        
        # Draw a rectangle around the recognized face and show the name
        if match_index == -1:
            name = "Unknown"
        else:
            name = known_face_names[match_index]
        cv2.rectangle(rgb_image, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(rgb_image, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        print(f"face recognized in {filename} as {name}")

    # Show the image with the recognized faces
    cv2_imshow(rgb_image)
    cv2.waitKey(0)

# Clean up
cv2.destroyAllWindows()